# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database **data.sqlite**.

In [1]:
#Your code here
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Display the names of all the employees in Boston.
Hint: join the employees and offices tables.

In [2]:
#Your code here
cur.execute("""SELECT lastName, firstName
               FROM employees
               JOIN offices
               USING(officeCode)
               WHERE city = 'Boston';""")

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,lastName,firstName
0,Firrelli,Julie
1,Patterson,Steve


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [3]:
#Your code here
# first, how many cities have offices
cur.execute("""SELECT city
               FROM offices
               GROUP BY city
               ORDER BY city
               ;""")

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df), 'total cities have offices')

# now to answer the question
cur.execute("""SELECT city, count(*)
               FROM offices
               JOIN employees
               USING(officeCode)
               GROUP BY city
               ORDER BY city
               ;""")

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df), 'cities have offices with employees')
df

7 total cities have offices
7 cities have offices with employees


,city,count(*)
0,Boston,2
1,London,2
2,NYC,2
3,Paris,5
4,San Francisco,6
5,Sydney,4
6,Tokyo,2


##### So, it seems there are no offices with zero employees.

## Write 3 Questions of your own and answer them

In [4]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold

#### Customers from How Many USA States have Placed Orders?

In [5]:
cur.execute("""SELECT count(distinct c.state) AS nbrStates, c.country
               FROM customers c
               JOIN orders o
               USING(customerNumber)
               WHERE c.country = 'USA'
               ;""")

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head(20)

,nbrStates,country
0,8,USA


#### What Product Received the Most Orders in the U.S.A. by State?

In [6]:
# Your code here

cur.execute("""SELECT p.productName, od.quantityOrdered, 
               c.state, c.country
               FROM products p
               JOIN orderdetails od
               USING(productCode)
               JOIN orders o
               USING(orderNumber)
               JOIN customers c
               USING(customerNumber)
               WHERE c.country = 'USA'
               GROUP BY c.state
               ORDER BY p.productName
               ;""")

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df), 'total records')
df

8 total records


,productName,quantityOrdered,state,country
0,1912 Ford Model T Delivery Wagon,40,NJ,USA
1,1917 Grand Touring Sedan,30,NH,USA
2,1917 Grand Touring Sedan,21,NV,USA
3,1952 Alpine Renault 1300,29,MA,USA
4,1958 Setra Bus,21,CA,USA
5,1965 Aston Martin DB5,42,CT,USA
6,1969 Harley Davidson Ultimate Chopper,30,NY,USA
7,2001 Ferrari Enzo,35,PA,USA


#### What were the Total Payments Recieved from Each of these States?

In [7]:
cur.execute("""SELECT sum(pt.amount) AS totalPayments, 
               c.state, c.country
               FROM payments pt
               JOIN customers c
               USING(customerNumber)
               WHERE c.country = 'USA'
               GROUP BY c.state
               ORDER BY totalPayments DESC
               ;""")

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df), 'total records')
df

8 total records


,totalPayments,state,country
0,1257258.60,CA,USA
1,562984.45,MA,USA
2,534928.45,NY,USA
3,234175.18,PA,USA
4,215771.06,CT,USA
5,116449.29,NH,USA
6,80180.98,NV,USA
7,38281.51,NJ,USA


## Level Up: Display the names of every individual product that each employee has sold

In [8]:
# Your code here
cur.execute("""SELECT e.lastName, e.firstName, p.productName
               FROM employees as e
               JOIN customers as c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders as o
               USING(customerNumber)
               JOIN orderdetails as od
               USING(orderNumber)
               JOIN products as p
               USING(productCode)
               GROUP BY e.employeeNumber, p.productName
               ORDER BY e.employeeNumber
               ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df), 'total records')
df.head(5)

1368 total records


,lastName,firstName,productName
0,Jennings,Leslie,18th Century Vintage Horse Carriage
1,Jennings,Leslie,18th century schooner
2,Jennings,Leslie,1900s Vintage Bi-Plane
3,Jennings,Leslie,1900s Vintage Tri-Plane
4,Jennings,Leslie,1903 Ford Model A


## Level Up: Display the Number of Products each employee has sold

In [9]:
# Your code here
cur.execute("""SELECT lastName, firstName, COUNT(productName)
               FROM employees emp
               JOIN customers cust                
               ON emp.employeeNumber = cust.salesRepEmployeeNumber
               JOIN orders 
               USING(customerNumber)
               JOIN orderdetails
               USING(orderNumber)                    
               JOIN products
               USING(productCode)
               GROUP BY lastName
               ORDER BY lastName""")

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df), 'total records')
df

15 total records


,lastName,firstName,COUNT(productName)
0,Bondur,Loui,177
1,Bott,Larry,236
2,Castillo,Pamela,272
3,Firrelli,Julie,124
4,Fixter,Andy,185
5,Gerard,Martin,114
6,Hernandez,Gerard,396
7,Jennings,Leslie,331
8,Jones,Barry,220
9,Marsh,Peter,185


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!